# Stance Detection for the Fake News Challenge

## Identifying Textual Relationships with Deep Neural Nets

### Check the problem context [here](https://drive.google.com/open?id=1KfWaZyQdGBw8AUTacJ2yY86Yxgw2Xwq0).

### Download files required for the project from [here](https://drive.google.com/open?id=10yf39ifEwVihw4xeJJR60oeFBY30Y5J8).

### The objective of this problem is around “stance detection,” which involves comparing a headline with a body of text from a news article to determine what relationship (if any) exists between the two.  

### There are 4 possible classifications:  
 
### 1. The article text agrees with the headline.   
### 2. The article text disagrees with the headline.  
### 3. The article text is a discussion of the headline, without taking a position on it.  
### 4. The article text is unrelated to the headline (i.e. it doesn’t address the same topic).  
  

In [ ]:
import numpy as np
import pandas as pd
from nltk import sent_tokenize
from keras.preprocessing.text import text_to_word_sequence
from sklearn.model_selection import train_test_split

## Step1: Load the given dataset  

1. Mount the google drive

2. Import Glove embeddings

3. Import the test and train datasets

### Loading the Glove Embeddings

In [ ]:
from zipfile import ZipFile
with ZipFile('glove.6B.zip', 'r') as z:
  z.extractall()

# Load the dataset [5 Marks]

1. Using [read_csv()](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_csv.html) in pandas load the given train datasets files **`train_bodies.csv`** and **`train_stances.csv`**

2. Using [merge](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.merge.html) command in pandas merge the two datasets based on the Body ID. 

Note: Save the final merged dataset in a dataframe with name **`dataset`**.

In [ ]:
bodies=pd.read_csv('train_bodies.csv')
stances=pd.read_csv('train_stances.csv')


In [ ]:
bodies.head()

,Body ID,articleBody
0,0,A small meteorite crashed into a wooded area i...
1,4,Last week we hinted at what was to come as Ebo...
2,5,(NEWSER) – Wonder how long a Quarter Pounder w...
3,6,"Posting photos of a gun-toting child online, I..."
4,7,At least 25 suspected Boko Haram insurgents we...


In [ ]:
stances.head()

,Headline,Body ID,Stance
0,Police find mass graves with at least '15 bodi...,712,unrelated
1,Hundreds of Palestinians flee floods in Gaza a...,158,agree
2,"Christian Bale passes on role of Steve Jobs, a...",137,unrelated
3,HBO and Apple in Talks for $15/Month Apple TV ...,1034,unrelated
4,Spider burrowed through tourist's stomach and ...,1923,disagree


In [ ]:
dataset=pd.merge(bodies, stances, left_on='Body ID', right_on='Body ID')


<h2> Check1:</h2>
  
<h3> You should see the below output if you run `dataset.head()` command as given below </h3>

In [ ]:
dataset.head()

,Body ID,articleBody,Headline,Stance
0,0,A small meteorite crashed into a wooded area i...,"Soldier shot, Parliament locked down after gun...",unrelated
1,0,A small meteorite crashed into a wooded area i...,Tourist dubbed ‘Spider Man’ after spider burro...,unrelated
2,0,A small meteorite crashed into a wooded area i...,Luke Somers 'killed in failed rescue attempt i...,unrelated
3,0,A small meteorite crashed into a wooded area i...,BREAKING: Soldier shot at War Memorial in Ottawa,unrelated
4,0,A small meteorite crashed into a wooded area i...,Giant 8ft 9in catfish weighing 19 stone caught...,unrelated


In [ ]:
dataset.head()

,Body ID,articleBody,Headline,Stance
0,0,A small meteorite crashed into a wooded area i...,"Soldier shot, Parliament locked down after gun...",unrelated
1,0,A small meteorite crashed into a wooded area i...,Tourist dubbed ‘Spider Man’ after spider burro...,unrelated
2,0,A small meteorite crashed into a wooded area i...,Luke Somers 'killed in failed rescue attempt i...,unrelated
3,0,A small meteorite crashed into a wooded area i...,BREAKING: Soldier shot at War Memorial in Ottawa,unrelated
4,0,A small meteorite crashed into a wooded area i...,Giant 8ft 9in catfish weighing 19 stone caught...,unrelated


## Step2: Data Pre-processing and setting some hyper parameters needed for model


#### Run the code given below to set the required parameters.

1. `MAX_SENTS` = Maximum no.of sentences to consider in an article.

2. `MAX_SENT_LENGTH` = Maximum no.of words to consider in a sentence.

3. `MAX_NB_WORDS` = Maximum no.of words in the total vocabualry.

4. `MAX_SENTS_HEADING` = Maximum no.of sentences to consider in a heading of an article.

In [ ]:
MAX_NB_WORDS = 20000
MAX_SENTS = 20
MAX_SENTS_HEADING = 1
MAX_SENT_LENGTH = 20
VALIDATION_SPLIT = 0.2

### Download the `Punkt` from nltk using the commands given below. This is for sentence tokenization.

For more info on how to use it, read [this](https://stackoverflow.com/questions/35275001/use-of-punktsentencetokenizer-in-nltk).



In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to C:\Users\Lenovo-
[nltk_data]     pc\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

# Tokenizing the text and loading the pre-trained Glove word embeddings for each token  [5 marks] 

Keras provides [Tokenizer API](https://keras.io/preprocessing/text/) for preparing text. Read it before going any further.

#### Import the Tokenizer from keras preprocessing text

In [ ]:
from keras.preprocessing.text import Tokenizer

#### Initialize the Tokenizer class with maximum vocabulary count as `MAX_NB_WORDS` initialized at the start of step2. 

In [ ]:
tk=Tokenizer(num_words=MAX_NB_WORDS, filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n', lower=True, split=' ', char_level=False, oov_token=None, document_count=0)





#### Now, using fit_on_texts() from Tokenizer class, lets encode the data 

Note: We need to fit articleBody and Headline also to cover all the words.

In [ ]:
tk.fit_on_texts(dataset['articleBody'])

In [ ]:
tk.fit_on_texts(dataset['Headline'])

In [ ]:
tk.word_docs

defaultdict(int,
            {'international': 5108,
             'only': 11736,
             '2014': 6319,
             'saballos': 346,
             'deep': 1088,
             'by': 38889,
             'is': 47076,
             'have': 38399,
             'i': 15682,
             'event': 3163,
             'house': 5898,
             'all': 19988,
             'news': 17193,
             'said': 34186,
             'that': 46407,
             'happened': 4211,
             'volcanologist': 332,
             'sky': 901,
             'we': 20215,
             'off': 12046,
             'scientists': 1322,
             'more': 22758,
             'with': 42484,
             'because': 13129,
             'which': 23501,
             'large': 3938,
             'rock': 999,
             'near': 8219,
             'the': 55248,
             'garcia': 297,
             'center': 3347,
             'pass': 1100,
             'left': 9084,
             'and': 50182,
             'has': 3976

In [ ]:
tk.word_counts

OrderedDict([('a', 478544),
             ('small', 5273),
             ('meteorite', 5848),
             ('crashed', 207),
             ('into', 25604),
             ('wooded', 554),
             ('area', 7132),
             ('in', 408093),
             ("nicaragua's", 526),
             ('capital', 6867),
             ('of', 441773),
             ('managua', 1833),
             ('overnight', 749),
             ('the', 1129192),
             ('government', 23653),
             ('said', 134456),
             ('sunday', 5416),
             ('residents', 3428),
             ('reported', 21926),
             ('hearing', 2514),
             ('mysterious', 853),
             ('boom', 1459),
             ('that', 234452),
             ('left', 11643),
             ('16', 2172),
             ('foot', 2441),
             ('deep', 1218),
             ('crater', 3683),
             ('near', 12938),
             ("city's", 726),
             ('airport', 5598),
             ('associated', 4582),
  

In [ ]:
tk.word_index

{'the': 1,
 'to': 2,
 'a': 3,
 'of': 4,
 'in': 5,
 'and': 6,
 'that': 7,
 'is': 8,
 'was': 9,
 'on': 10,
 'for': 11,
 'said': 12,
 'he': 13,
 'with': 14,
 'it': 15,
 'his': 16,
 'have': 17,
 'as': 18,
 'by': 19,
 'has': 20,
 'at': 21,
 'from': 22,
 'be': 23,
 'an': 24,
 'not': 25,
 'are': 26,
 'been': 27,
 '”': 28,
 'but': 29,
 'who': 30,
 'this': 31,
 'had': 32,
 'they': 33,
 'after': 34,
 'i': 35,
 'were': 36,
 'we': 37,
 'will': 38,
 'about': 39,
 'one': 40,
 'or': 41,
 'isis': 42,
 'video': 43,
 'which': 44,
 'she': 45,
 'apple': 46,
 'state': 47,
 'up': 48,
 'her': 49,
 'would': 50,
 'their': 51,
 'more': 52,
 'also': 53,
 'when': 54,
 'told': 55,
 'new': 56,
 'out': 57,
 'no': 58,
 'all': 59,
 'people': 60,
 'there': 61,
 'you': 62,
 'its': 63,
 'if': 64,
 'him': 65,
 'islamic': 66,
 'news': 67,
 'man': 68,
 'could': 69,
 'what': 70,
 'year': 71,
 'watch': 72,
 'time': 73,
 'al': 74,
 'over': 75,
 'some': 76,
 'group': 77,
 'according': 78,
 'u': 79,
 's': 80,
 'into': 81,
 'firs

In [ ]:
tk.word_index.get('the')

1

In [ ]:
tk.document_count

99944

#### fit_on_texts() gives the following attributes in the output as given [here](https://faroit.github.io/keras-docs/1.2.2/preprocessing/text/).

* **word_counts:** dictionary mapping words (str) to the number of times they appeared on during fit. Only set after fit_on_texts was called.

* **word_docs:** dictionary mapping words (str) to the number of documents/texts they appeared on during fit. Only set after fit_on_texts was called.

* **word_index:** dictionary mapping words (str) to their rank/index (int). Only set after fit_on_texts was called.

* **document_count:** int. Number of documents (texts/sequences) the tokenizer was trained on. Only set after fit_on_texts or fit_on_sequences was called.



### Now, tokenize the sentences using nltk sent_tokenize() and encode the senteces with the ids we got form the above `t.word_index`

Initialise 2 lists with names `texts` and `articles`.

```
texts = [] to store text of article as it is.

articles = [] split the above text into a list of sentences.
```

In [ ]:
texts=[]
articles=[]
for row in dataset['articleBody']:
    #print(row)
    texts.append(row)
    articles.append(sent_tokenize(row))

## Check 2:

first element of texts and articles should be as given below. 

In [ ]:
texts[0]

'A small meteorite crashed into a wooded area in Nicaragua\'s capital of Managua overnight, the government said Sunday. Residents reported hearing a mysterious boom that left a 16-foot deep crater near the city\'s airport, the Associated Press reports. \n\nGovernment spokeswoman Rosario Murillo said a committee formed by the government to study the event determined it was a "relatively small" meteorite that "appears to have come off an asteroid that was passing close to Earth." House-sized asteroid 2014 RC, which measured 60 feet in diameter, skimmed the Earth this weekend, ABC News reports. \nMurillo said Nicaragua will ask international experts to help local scientists in understanding what happened.\n\nThe crater left by the meteorite had a radius of 39 feet and a depth of 16 feet,  said Humberto Saballos, a volcanologist with the Nicaraguan Institute of Territorial Studies who was on the committee. He said it is still not clear if the meteorite disintegrated or was buried.\n\nHumbe

In [ ]:
texts[0]

'A small meteorite crashed into a wooded area in Nicaragua\'s capital of Managua overnight, the government said Sunday. Residents reported hearing a mysterious boom that left a 16-foot deep crater near the city\'s airport, the Associated Press reports. \n\nGovernment spokeswoman Rosario Murillo said a committee formed by the government to study the event determined it was a "relatively small" meteorite that "appears to have come off an asteroid that was passing close to Earth." House-sized asteroid 2014 RC, which measured 60 feet in diameter, skimmed the Earth this weekend, ABC News reports. \nMurillo said Nicaragua will ask international experts to help local scientists in understanding what happened.\n\nThe crater left by the meteorite had a radius of 39 feet and a depth of 16 feet,  said Humberto Saballos, a volcanologist with the Nicaraguan Institute of Territorial Studies who was on the committee. He said it is still not clear if the meteorite disintegrated or was buried.\n\nHumbe

In [ ]:
articles[0]

["A small meteorite crashed into a wooded area in Nicaragua's capital of Managua overnight, the government said Sunday.",
 "Residents reported hearing a mysterious boom that left a 16-foot deep crater near the city's airport, the Associated Press reports.",
 'Government spokeswoman Rosario Murillo said a committee formed by the government to study the event determined it was a "relatively small" meteorite that "appears to have come off an asteroid that was passing close to Earth."',
 'House-sized asteroid 2014 RC, which measured 60 feet in diameter, skimmed the Earth this weekend, ABC News reports.',
 'Murillo said Nicaragua will ask international experts to help local scientists in understanding what happened.',
 'The crater left by the meteorite had a radius of 39 feet and a depth of 16 feet,  said Humberto Saballos, a volcanologist with the Nicaraguan Institute of Territorial Studies who was on the committee.',
 'He said it is still not clear if the meteorite disintegrated or was bu

# Now iterate through each article and each sentence to encode the words into ids using t.word_index  [5 marks] 

Here, to get words from sentence you can use `text_to_word_sequence` from keras preprocessing text.

1. Import text_to_word_sequence

2. Initialize a variable of shape (no.of articles, MAX_SENTS, MAX_SENT_LENGTH) with name `data` with zeros first (you can use numpy [np.zeros](https://docs.scipy.org/doc/numpy/reference/generated/numpy.zeros.html) to initialize with all zeros)and then update it while iterating through the words and sentences in each article.

In [ ]:
data=np.zeros([len(articles),MAX_SENTS, MAX_SENT_LENGTH],dtype=int)
len(data)

49972

In [ ]:
data.shape

(49972, 20, 20)

In [ ]:
data[1,1,1]

0

In [ ]:
a=int(0)
s=int(0)
w=int(0)
for art in articles:
    for sent in art:
        for wrd in text_to_word_sequence(sent):
            wd=str(wrd)
            if ( w < 20 and s < 20):
                data[a,s,w]=tk.word_index.get(wd)
            w=w+1
        s=s+1
        w=int(0)
    a = a+1
    s=int(0)
    

### Check 3:

Accessing first element in data should give something like given below.

In [ ]:
data[0, :, :]

array([[    3,   481,   427,  7211,    81,     3,  3733,   331,     5,
         3891,   350,     4,  1431,  2958,     1,    89,    12,   464,
            0,     0],
       [  758,    95,  1047,     3,  2679,  1752,     7,   189,     3,
         1217,  1075,  2030,   700,   159,     1,  3032,   448,     1,
          555,   235],
       [   89,  1067,  4115,  2349,    12,     3,  1092,  3306,    19,
            1,    89,     2,  1793,     1,   521,  2009,    15,     9,
            3,  3111],
       [  181,  3640,   972,   200,  2556,    44,  6775,  1722,  1252,
            5, 13317, 17936,     1,   778,    31,   740,  3990,    67,
           85,     0],
       [ 2349,    12,  1557,    38,  1094,   351,   775,     2,   367,
          260,  1770,     5,  4450,    70,   494,     0,     0,     0,
            0,     0],
       [    1,   700,   189,    19,     1,   427,    32,     3,  7417,
            4,  2159,  1252,     6,     3,  5270,     4,  1217,  1252,
           12,  3363],
       [  

In [ ]:
data[0, :, :]

array([[    3,   481,   427,  7211,    81,     3,  3733,   331,     5,
         3891,   350,     4,  1431,  2958,     1,    89,    12,   464,
            0,     0],
       [  758,    95,  1047,     3,  2679,  1752,     7,   189,     3,
         1217,  1075,  2030,   700,   159,     1,  3032,   448,     1,
          555,   235],
       [   89,  1067,  4115,  2349,    12,     3,  1092,  3306,    19,
            1,    89,     2,  1793,     1,   521,  2009,    15,     9,
            3,  3111],
       [  181,  3640,   972,   200,  2556,    44,  6775,  1722,  1252,
            5, 13317, 17936,     1,   778,    31,   740,  3990,    67,
           85,     0],
       [ 2349,    12,  1557,    38,  1094,   351,   775,     2,   367,
          260,  1770,     5,  4450,    70,   494,     0,     0,     0,
            0,     0],
       [    1,   700,   189,    19,     1,   427,    32,     3,  7417,
            4,  2159,  1252,     6,     3,  5270,     4,  1217,  1252,
           12,  3363],
       [  

# Repeat the same process for the `Headings` as well. Use variables with names `texts_heading` and `articles_heading` accordingly. [5 marks] 

In [ ]:
texts_heading=[]
articles_heading=[]
for row in dataset['Headline']:
    #print(row)
    texts_heading.append(row)
    articles_heading.append(sent_tokenize(row))

In [ ]:
data_heading=np.zeros([len(articles_heading), MAX_SENT_LENGTH],dtype=int)
len(data_heading)

49972

In [ ]:
# Headlines contain only one sentence hence no need for sentence loop
w=int(0)
a=int(0)
for art in articles_heading:
    #print('hello')
    for wrd in text_to_word_sequence(art[0]):
        #print(wrd)
        wd=str(wrd)
        if ( w < 20):
            data_heading[a,w]=tk.word_index.get(wd)
        w=w+1
    w=int(0)
    a=a+1

In [ ]:
articles_heading[0]

['Soldier shot, Parliament locked down after gunfire erupts at war memorial']

In [ ]:
data_heading[0, :]

array([  718,   206,   343,  7134,   193,    34,  1338, 11554,    21,
         233,   686,     0,     0,     0,     0,     0,     0,     0,
           0,     0])

### Now the features are ready, lets make the labels ready for the model to process.

### Convert labels into one-hot vectors

You can use [get_dummies](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html) in pandas to create one-hot vectors.

In [ ]:
labels=pd.get_dummies(dataset['Stance'])

### Check 4:

The shape of data and labels shoould match the given below numbers.

In [ ]:
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)

Shape of data tensor: (49972, 20, 20)
Shape of label tensor: (49972, 4)


In [ ]:
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)

Shape of data tensor: (49972, 20, 20)
Shape of label tensor: (49972, 4)


### Shuffle the data

In [ ]:
## get numbers upto no.of articles
indices = np.arange(data.shape[0])
## shuffle the numbers
np.random.shuffle(indices)

In [ ]:
indices.shape

(49972,)

In [ ]:
len(labels)

49972

In [ ]:
## shuffle the data
data = data[indices]
data_heading = data_heading[indices]
## shuffle the labels according to data
labels = labels.iloc[indices]

# Try the sequential model approach and report the accuracy score. [10 marks]  

## Merging the body and headline into one Dataframe to be passed to the model

In [ ]:
from itertools import chain
data_fin=[]
for lst in data:
    data_fin.append(list(chain.from_iterable(lst)))

In [ ]:
data_fin_df=pd.DataFrame(data_fin)

In [ ]:
data_fin_df.shape

(49972, 400)

In [ ]:
data_heading_df=pd.DataFrame(data_heading)

In [ ]:
data_heading_df.shape

(49972, 20)

In [ ]:
data_fin=pd.concat([data_fin_df, data_heading_df], axis=1)

In [ ]:
data_fin.head()

,0,1,2,3,4,5,6,7,8,9,...,10,11,12,13,14,15,16,17,18,19
0,1,484,77,42,20,389,814,106,153,202,...,0,0,0,0,0,0,0,0,0,0
1,75,1,740,1013,2697,3994,3303,894,2,3,...,0,0,0,0,0,0,0,0,0,0
2,66,47,122,190,3,43,10,209,3124,2,...,9039,0,0,0,0,0,0,0,0,0
3,439,1203,1956,3,819,4,1,395,2458,3048,...,0,0,0,0,0,0,0,0,0,0
4,7604,5790,10,269,2142,1355,7,1,312,966,...,148,29,26,33,7073,65,14,403,8696,1321


In [ ]:
cols=np.arange(420)

In [ ]:
data_fin.shape

(49972, 420)

In [ ]:
data_fin.columns=cols

In [ ]:
data_fin.head()

,0,1,2,3,4,5,6,7,8,9,...,410,411,412,413,414,415,416,417,418,419
0,1,484,77,42,20,389,814,106,153,202,...,0,0,0,0,0,0,0,0,0,0
1,75,1,740,1013,2697,3994,3303,894,2,3,...,0,0,0,0,0,0,0,0,0,0
2,66,47,122,190,3,43,10,209,3124,2,...,9039,0,0,0,0,0,0,0,0,0
3,439,1203,1956,3,819,4,1,395,2458,3048,...,0,0,0,0,0,0,0,0,0,0
4,7604,5790,10,269,2142,1355,7,1,312,966,...,148,29,26,33,7073,65,14,403,8696,1321


In [ ]:
labels_df=pd.DataFrame(labels)

### Split into train and validation sets. Split the train set 80:20 ratio to get the train and validation sets.


Use the variable names as given below:

x_train, x_val - for body of articles.

x-heading_train, x_heading_val - for heading of articles.

y_train - for training labels.

y_val - for validation labels.



In [ ]:
X_train, X_val, y_train, y_val = train_test_split(data_fin,labels_df, test_size=0.2)

In [ ]:
X_train.shape

(39977, 420)

In [ ]:
y_train.shape

(39977, 4)

In [ ]:
X_val.shape

(9995, 420)

### Create embedding matrix with the glove embeddings


Run the below code to create embedding_matrix which has all the words and their glove embedding if present in glove word list.

In [ ]:
# load the whole embedding into memory
embeddings_index = dict()
f = open('./glove.6B.100d.txt', encoding="utf8")
for line in f:
	values = line.split()
	word = values[0]
	coefs = np.asarray(values[1:], dtype='float32')
	embeddings_index[word] = coefs
f.close()
print('Loaded %s word vectors.' % len(embeddings_index))

# create a weight matrix for words in training docs
embedding_matrix = np.zeros((len(tk.word_index), 100))


for word, i in tk.word_index.items():
	embedding_vector = embeddings_index.get(word)
	if embedding_vector is not None:
		embedding_matrix[i] = embedding_vector

Loaded 400000 word vectors.


In [ ]:
len(tk.word_index)

27873

# Import Model Layers

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Embedding, Flatten
from keras.preprocessing.sequence import pad_sequences

### Model

In [ ]:
model=Sequential()
e = Embedding(27873, 100, weights=[embedding_matrix], input_length=420, trainable=False)
model.add(e)
model.add(Flatten())
model.add(Dense(4, activation='softmax'))
# compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
# summarize the model
print(model.summary())


W0101 10:33:54.352883  5316 deprecation_wrapper.py:119] From C:\Users\Lenovo-pc\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0101 10:33:55.344933  5316 deprecation_wrapper.py:119] From C:\Users\Lenovo-pc\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0101 10:33:55.642208  5316 deprecation_wrapper.py:119] From C:\Users\Lenovo-pc\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0101 10:33:55.923612  5316 deprecation_wrapper.py:119] From C:\Users\Lenovo-pc\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W0101 10:33:55.925614  5316 deprecation_

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 420, 100)          2787300   
_________________________________________________________________
flatten_1 (Flatten)          (None, 42000)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 4)                 168004    
Total params: 2,955,304
Trainable params: 168,004
Non-trainable params: 2,787,300
_________________________________________________________________
None


### Compile and fit the model

In [ ]:
# fit the model
model.fit(X_train, y_train, epochs=70, verbose=0)
# evaluate the model
loss, accuracy = model.evaluate(X_val, y_val, verbose=0)
print('Accuracy: %f' % (accuracy*100))

Accuracy: 91.418209
